In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\projects\\Potato-disease-end-to-end\\research'

In [2]:
import os

os.chdir("../")
%pwd

'c:\\Users\\44787\\Desktop\\projects\\Potato-disease-end-to-end'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainingConfig:
    root_dir: Path
    dataset_path: Path
    model_save: Path
    BATCH_SIZE: int
    IMAGE_SIZE: int
    CHANNELS: int
    EPOCHS: int

In [1]:
from src.constants import PARAMS_PATH, CONFIG_PATH
from src.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config=CONFIG_PATH,
                 params=PARAMS_PATH) -> None:
        self.config = read_yaml(config)
        self.params = read_yaml(params)

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            dataset_path=config.dataset_path,
            model_save=config.model_save,
            BATCH_SIZE=self.params.BATCH_SIZE,
            IMAGE_SIZE=self.params.IMAGE_SIZE,
            CHANNELS=self.params.CHANNELS,
            EPOCHS=self.params.EPOCHS,
        )
        return model_training_config

In [2]:
from src.logging import logger
import tensorflow as tf
import mlflow
from urllib.parse import urlparse

[2023-12-23 21:38:39,148: WARNING: module_wrapper: From c:\Users\44787\anaconda3\envs\potato-env\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [7]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/fraidoon_omarzai/Potato-disease-end-to-end.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="fraidoon_omarzai"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bc25b16bd5206328d8899cf34377f26ad71d1420"

In [10]:
class ModelTraining:
    def __init__(self, config):
        self.config = config

    def split_dataset(self,
                      dataset,
                      train_split=0.8,
                      test_split=0.1,
                      val_split=0.1,
                      shuffle=True):

        assert (train_split+test_split+val_split) == 1

        dataset_size = len(dataset)
        if shuffle:
            dataset = dataset.shuffle(10000, seed=42)

        train_size = int(train_split * dataset_size)
        val_size = int(val_split * dataset_size)

        train_data = dataset.take(train_size)
        val_data = dataset.skip(train_size).take(val_size)
        test_data = dataset.skip(train_size).skip(val_size)

        return train_data, val_data, test_data

    def prepare_dataset(self):
        dataset = tf.keras.preprocessing.image_dataset_from_directory(
            self.config.dataset_path,
            image_size=(self.config.IMAGE_SIZE, self.config.IMAGE_SIZE),
            batch_size=self.config.BATCH_SIZE,
            shuffle=True
        )
        logger.info(f'name of classes: {dataset.class_names}')

        train_data, val_data, test_data = self.split_dataset(dataset)

        train_data = train_data.cache().shuffle(
            1000).prefetch(buffer_size=tf.data.AUTOTUNE)
        val_data = val_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
        test_data = test_data.cache().shuffle(
            1000).prefetch(buffer_size=tf.data.AUTOTUNE)

        # applay data agumentation to train dataset
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.RandomFlip(
                "horizontal_and_vertical"),
            tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
        ])
        train_data = train_data.map(lambda x, y:
                                    (data_augmentation(x, training=True), y)).prefetch(buffer_size=tf.data.AUTOTUNE)

        return train_data, val_data, test_data

    def training(self):
        resize_and_rescale = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.Resizing(
                self.config.IMAGE_SIZE, self.config.IMAGE_SIZE),
            tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
        ])

        # create a model architecture
        input_shape = (self.config.BATCH_SIZE, self.config.IMAGE_SIZE,
                       self.config.IMAGE_SIZE, self.config.CHANNELS)
        n_classes = 3

        
        model = tf.keras.models.Sequential([
            resize_and_rescale,
            tf.keras.layers.Conv2D(32, kernel_size=(
                3, 3), activation='relu', input_shape=input_shape),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64,  kernel_size=(3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64,  kernel_size=(3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(n_classes, activation='softmax'),
        ])
        model.build(input_shape=input_shape)
        logger.info(model.summary())

        # compile the model
        model.compile(
            optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=False),
            metrics=['accuracy']
        )

        train_data, val_data, test_data = self.prepare_dataset()

        mlflow.set_experiment('Tensorflow Models')
        mlflow.set_registry_uri('https://dagshub.com/fraidoon_omarzai/Potato-disease-end-to-end.mlflow')
        
        with mlflow.start_run(run_name="Tf") as mlops_run:
        
            # train the model
            model.fit(
                train_data,
                batch_size=self.config.BATCH_SIZE,
                validation_data=val_data,
                verbose=1,
                epochs=self.config.EPOCHS
            )
            
            # evaluate the model
            results = model.evaluate(test_data)
            logger.info(f"Model evaluation: {results}")
            
            mlflow.log_param('batch size', self.config.BATCH_SIZE)
            mlflow.log_param('epochs', self.config.EPOCHS)
            
            mlflow.log_metric('accuracy', results[1])
            
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.tensorflow.log_model(model, "model", registered_model_name="TFModel")
            else:
                mlflow.tensorflow.log_model(model, "model")
        

    #     self.save_model(
    #         self.config.model_save,
    #         model
    #     )

    # @staticmethod
    # def save_model(path: Path, model: tf.keras.Model):
    #     model.save(path)

In [11]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(model_training_config)
    model_training.training()
except Exception as e:
    raise e

[2023-12-18 10:56:34,367: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-18 10:56:34,374: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-18 10:56:34,377: INFO: common: created directory at: artifacts/model_training]
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d_6 (Conv2D)           (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (32, 127, 127, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (32, 125, 125, 64)        18496     
                                                       

2023/12/18 10:56:35 INFO mlflow.tracking.fluent: Experiment with name 'Tensorflow Models' does not exist. Creating a new experiment.


Epoch 1/2
54/54 [==============================] - 35s 564ms/step - loss: 0.9138 - accuracy: 0.4953 - val_loss: 0.8136 - val_accuracy: 0.6198
Epoch 2/2
8/8 [==============================] - 2s 126ms/step - loss: 0.8310 - accuracy: 0.5781
[2023-12-18 10:57:40,106: INFO: 2247739627: Model evaluation: [0.8309705257415771, 0.578125]]


2023/12/18 10:57:40 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-12-18 10:57:42,917: INFO: builder_impl: Assets written to: C:\Users\44787\AppData\Local\Temp\tmp7tkx6mf0\model\data\model\assets]


Registered model 'TFModel' already exists. Creating a new version of this model...
2023/12/18 10:58:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: TFModel, version 2
Created version '2' of model 'TFModel'.
